In [ ]:
import copy
import glob
import japanize_matplotlib
import math
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import pandas as pd
import random
import sys
from sklearn import linear_model
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as sp
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
# 平均絶対パーセント誤差 (MAPE)(Mean Absolute Percent Error (MAPE))を返す関数
# 引数として長さの同じ二つのリストをとる
# 引数l1: 実測値のリスト
# 引数l2: 予測値のリスト
# 単位：％

def returnMapeScore(l1, l2):
    return_num = 0
    if(len(l1) != len(l2)):
        print("引数のリストの長さが異なります", end=", ")
        return -1
    for i in range(len(l1)):
        l1_num = l1[i]
        l2_num = l2[i]

        return_num += abs((l1_num - l2_num)/l1_num)

    return_num /= len(l1)
    return_num *= 100
    return return_num


# 使用例：returnMapeScore([1,2,3,4], [4,3,2,1])
type(returnMapeScore([1, 2, 3, 4], [4, 3, 2, 1]))


def test_returnMapeScore():
    l1 = [1, 2, 3, 4]
    l2 = [4, 3, 2, 1]
    ansByFunc = returnMapeScore(l1, l2)
    ansByHand = (abs(1-4)/1 + abs(2-3)/2 + abs(3-2)/3 + abs(4-1)/4)/4 * 100
    # 多少の誤差を許容する
    ansByFunc = int(ansByFunc * 100) / 100
    ansByHand = int(ansByHand * 100) / 100

    assert ansByFunc == ansByHand

In [ ]:
# ベンチマークを指定して存在するファイル名のものを返す
def returnExistingFileNames(benchmarkNames=[], classes=[], processes=[], csvDirPath="./csv_files"):
    candidateFileNames = []
    returnList = []
    for benchmarkName in benchmarkNames:
        for benchmarkClass in classes:
            for process in processes:
                candidateFileNames.append(
                    f"pprof_{benchmarkName}{benchmarkClass}{process}.csv")
    for candidateFileName in candidateFileNames:
        filePath = os.path.join(csvDirPath, candidateFileName)
#         print(filePath)
        if(os.path.exists(filePath) and os.stat(filePath).st_size != 0):
            returnList.append(candidateFileName)
    print(returnList)
    return(returnList)


def test_returnExistingFileNames():
    benchmarkNames = ["test"]
    classes = ["A", "B", "C", "D"]
    processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    csvDirPath = "../csv_files/"
    returnedList = returnExistingFileNames(
        benchmarkNames=benchmarkNames, classes=classes, processes=processes, csvDirPath=csvDirPath)
    print(returnedList)
    assert returnedList[0] == "pprof_testA128.csv"
    assert returnedList[1] == "pprof_testB256.csv"

In [ ]:
# ベンチマーク名・プロセス数・ベンチマーククラスをリストで渡して、実在するデータが集計されたDFを返す
def returnCollectedExistingData(benchmarkNames=[], classes=[], processes=[], csvDirPath="./csv_files"):
    

def test_returnCollectedExistingData():
    benchmarkNames = ["test"]
    classes = ["A", "B", "C", "D"]
    processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    csvDirPath = "../csv_files/"
    returnedDF = returnCollectedExistingData(
        benchmarkNames=benchmarkNames, classes=classes, processes=processes, csvDirPath=csvDirPath)

    case01 = {"benchmarkName": "test", "benchmarkClass": "A", "process": 128, "functionCalls" : {"function00": 99, "function01": 77, "function02": 555}}
    case02 = {"benchmarkName": "test", "benchmarkClass": "B", "process": 256, "functionCalls" : {"function00": 5, "function01": 70, "function02": 900}}

    for case in [case01, case02]:
        benchmarkName = case["benchmarkName"]
        benchmarkClass = case["benchmarkClass"]
        process = case["process"]
        for functionName in case["functionCalls"]:
            functionCallNum = case["functionCalls"][functionName]
            targetData = returnedData[(returnedData['benchmarkNames'] == benchmarkName) & (returnedData['benchmarkClass'] == benchmarkClass) & (
                returnedData["process"] == process) & (returnedData["functionName"] == functionName)]
            assert targetData.loc['Bob', 'age'] == functionCallNum